# **engagio:** Engagement Model

### **File Handling Cell**

In [ ]:
import os
import random

import cv2
import torch

from utility import *
from architecture import SignalEncoder
from imbalance import SelfAdjDiceLoss

random.seed(2147483647)
torch.manual_seed(2147483647)

PATH = 'DAiSEE/DataSet/'

FPS = 30
BATCH_SIZE = 16
VIDEO_LENGTH = 10
FRAME_INTERVAL = 2

### **Visual Features Extraction**

In [ ]:
# paste in your code here @AlakhsimarSingh

# fan_in_A = number of input features
fan_in_A = 0    # replace 0 with the actual value

### **Physiological Features Extraction**

In [ ]:
# paste in your code here @NischayVerma

# fan_in_N = number of input features
fan_in_N = 0    # replace 0 with the actual value

### **Device Auto Detection**

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
print(device)

### **Model Definition**

In [ ]:
class EngagementModel(torch.nn.Module):

    def __init__(self, fan_in_A, fan_in_N, fan_out_A, fan_out_N, fan_out, n_embd, head_size, n_heads, n_blocks, ffwd_mul):
        super().__init__()
        self.fan_out = fan_out
        self.signal_encoder_A = SignalEncoder(fan_in_A, fan_out_A, n_embd, head_size, n_heads, n_blocks, ffwd_mul)
        self.signal_encoder_N = SignalEncoder(fan_in_N, fan_out_N, n_embd, head_size, n_heads, n_blocks, ffwd_mul)
        self.lm_head = torch.nn.Sequential(
                            torch.nn.Linear(fan_out_A + fan_out_N, fan_out),
                            torch.nn.GELU()
                        )

    def forward(self, x_A, x_N, idx=None, weighted=False):
        x_A = self.signal_encoder_A(x_A)
        x_N = self.signal_encoder_N(x_N)
        
        out = torch.cat([x_A, x_N], -1)
        out = self.lm_head(out)
        B, T, C = out.shape

        loss = None
        if idx is not None:
            x = out.float()
            y = idx.unsqueeze(1).repeat(1, T, 1).float()
            x = x.view(-1, self.fan_out)
            y = y.view(-1).long()
            criterion = SelfAdjDiceLoss() if weighted else torch.nn.CrossEntropyLoss()
            loss = criterion(x, y)
            # loss = torch.nn.functional.cross_entropy(x, y, weight=torch.tensor([1/61, 1/455, 1/4422, 1/3987], dtype=x.dtype, device=x.device) if weighted else None)
        
        return out, loss

# NOTE: n_embd = head_size * n_heads
model = EngagementModel(fan_in_A, fan_in_N, 4, 4, 4, 64, 16, 4, 8, 4)
model.to(device)

sum(p.nelement() for p in model.parameters())

### **Training Loop**

In [ ]:
def compute_accuracy(logits, true_labels):
    logits = logits.to(device)
    true_labels = true_labels.to(device)

    B, T, C = logits.shape
    logits = logits.view(-1, C)
    probabilities = torch.nn.functional.softmax(logits, dim=-1)  
    
    predicted_classes = torch.argmax(probabilities, dim=-1)

    predicted_classes = predicted_classes.view(B, T, 1)
    true_labels = true_labels.unsqueeze(1).repeat(1, T, 1)
    
    correct_predictions = (predicted_classes == true_labels).float().sum()
    total_predictions = B*T
    accuracy = correct_predictions / total_predictions
    
    return accuracy.item()

In [ ]:
acci = []
lossi = []

lr_i = 1e-3
lr_f = 1e-6
iters = 100

for i in range(iters):
    lr = 1e-3 + ((lr_f - lr_i)/iters)*i
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    X_batch, Y_batch = load_data(PATH, 'Train', BATCH_SIZE)

    # ----------------------------------------------------------------------------------------------------------

    # code for X_batch_A, X_batch_N
    X_batch_A = X_batch                 # replace X_batch_A with the actual code @AlakhsimarSingh
    X_batch_N = X_batch                 # replace X_batch_N with the actual code @NischayVerma
    X_batch_A.requires_grad = True    
    X_batch_N.requires_grad = True    
    
    logits, loss = model(X_batch_A.to(device), X_batch_N.to(device), Y_batch.to(device), weighted=True)

    # ----------------------------------------------------------------------------------------------------------

    acc = compute_accuracy(logits, Y_batch)
    print(f'{i} iteration, loss: {loss:.4f}, acc: {acc:.4f}')

    acci.append(acc)
    lossi.append(loss.item())

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f'Average loss: {sum(lossi)/len(lossi):.4f}, Average accuracy: {sum(acci)/len(acci):.4f}')

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.plot(lossi, color='blue')
ax1.set_title('Loss Plot')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Lossi Values')

ax2.plot(acci, color='green')
ax2.set_title('Acci Plot')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy Values')

plt.tight_layout()
plt.show()

### **Metrics on Validation Data**

In [ ]:
acci = []
lossi = []
for i in range(10):
    X_batch, Y_batch = load_data(PATH, 'Validation', BATCH_SIZE)
    
    # ----------------------------------------------------------------------------------------------------------

    # code for X_batch_A, X_batch_N
    X_batch_A = X_batch                 # replace X_batch_A with the actual code @AlakhsimarSingh
    X_batch_N = X_batch                 # replace X_batch_N with the actual code @NischayVerma
    X_batch_A.requires_grad = True    
    X_batch_N.requires_grad = True    
    
    logits, loss = model(X_batch_A.to(device), X_batch_N.to(device), Y_batch.to(device), weighted=True)

    # ----------------------------------------------------------------------------------------------------------

    logits, loss = model(X_batch.to(device), X_batch.to(device), Y_batch.to(device))

    acc = compute_accuracy(logits, Y_batch)

    acci.append(acc)
    lossi.append(loss.item())

print(f'Average loss: {sum(lossi)/len(lossi):.4f}, Average accuracy: {sum(acci)/len(acci):.4f}')

### **Late Fusion Techniques**

In [ ]:
# @AlakhsimarSingh ML code starts here

In [ ]:
# @NischayVerma ML code starts here

In [ ]:
class LateFusion(torch.nn.Module):
    def __init__(self, num_modalities, num_classses, n_embd):
        super().__init__()

        self.num_classes = num_classses
        self.num_modalities = num_modalities
        self.wei = torch.nn.Sequential(
            torch.nn.Linear(self.num_modalities, 1),
            torch.nn.ReLU()
        )

    def forward(self, logits, y=None):
        out = self.wei(logits)
        out = out.squeeze(-1)
        loss = None
        if y is not None:
            x = out.float()
            y = y.long()
            loss = torch.nn.functional.cross_entropy(x, y)
        return out, loss
    
model = LateFusion(2, 4, 64)
model.to(device)

### **Training Loop**

In [ ]:
def _compute_accuracy(logits, true_labels):
    logits = logits.to(device)
    true_labels = true_labels.to(device)
    
    B, C = logits.shape
    logits = logits.view(-1, C)
    probabilities = torch.nn.functional.softmax(logits, dim=-1)
    
    predicted_classes = torch.argmax(probabilities, dim=-1)
    
    correct_predictions = (predicted_classes == true_labels).float().sum()
    total_predictions = B
    accuracy = correct_predictions / total_predictions
    
    return accuracy.item()

In [ ]:
acci = []
lossi = []

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for i in range(1000):
    X_batch, Y_batch = load_data(PATH, 'Train', BATCH_SIZE)

    # ------------------------------------------
    # add your signals here
    X_batch_A = X_batch     # @AlakhsimarSingh signals
    X_batch_N = X_batch     # @NischayVerma signals
    # ------------------------------------------

    X_batch = torch.stack([X_batch_A, X_batch_N], dim=-1)
    logits, loss = model(X_batch.to(device), Y_batch.to(device))

    acc = _compute_accuracy(logits, Y_batch)
    print(f'{i} iteration, loss: {loss.item():.4f}, acc: {acc:.4f}')

    lossi.append(loss.item())
    acci.append(acc)

    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(f'Average loss: {sum(lossi)/len(lossi):.4f}, Average accuracy: {sum(acci)/len(acci):.4f}')

In [ ]:
import matplotlib.pyplot as plt
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ax1.plot(lossi, color='blue')
ax1.set_title('Loss Plot')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Lossi Values')

ax2.plot(acci, color='green')
ax2.set_title('Acci Plot')
ax2.set_xlabel('Epochs')
ax2.set_ylabel('Accuracy Values')

plt.tight_layout()
plt.show()

In [ ]:
acci = []
lossi = []

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for i in range(1000):
    X_batch, Y_batch = load_data(PATH, 'Validation', BATCH_SIZE)

    # ------------------------------------------
    # add your signals here
    X_batch_A = X_batch     # @AlakhsimarSingh signals
    X_batch_N = X_batch     # @NischayVerma signals
    # ------------------------------------------

    X_batch = torch.stack([X_batch_A, X_batch_N], dim=-1)
    logits, loss = model(X_batch.to(device), Y_batch.to(device))

    acc = _compute_accuracy(logits, Y_batch)

    lossi.append(loss.item())
    acci.append(acc)

print(f'Average loss: {sum(lossi)/len(lossi):.4f}, Average accuracy: {sum(acci)/len(acci):.4f}')